In [1]:
import pandas as pd
import dateutil.relativedelta
import quandl
import sys
import pickle
from sklearn.externals import joblib

In [2]:
quandl.ApiConfig.api_key = "bkLgy-fmbYDf_AuKMJeV"
    

In [3]:
sp_test = pd.read_csv("GSPC.csv")

In [4]:
column_not_needed = ["Open","High","Low","Adj Close","Volume"]

In [5]:
sp_test.drop(column_not_needed,axis=1,inplace=True)

In [6]:
sp_test.columns = ['datekey','sp_price']

In [7]:
sp_test['datekey'] = pd.to_datetime(sp_test['datekey'])

In [8]:
stck_fund_attr = ['datekey','DE','PE1','PS1','PB','NETMARGIN',
                                        'MARKETCAP',
                                        'EV',
                                        'EVEBITDA',
                                        'REVENUEUSD',
                                        'GP',
                                        'EBITDA',
                                        'NETINCCMNUSD',
                                        'EPSDIL',
                                        'DEBTUSD',
                                        'CURRENTRATIO',
                                        'BVPS',
                                        'NCFO',
                                        'DEPAMOR',
                                        'SBCOMP',
                                        'NCFI',
                                        'CAPEX',
                                        'NCFBUS',
                                        'NCFINV',
                                        'NCFF',
                                        'NCFDEBT',
                                        'NCFCOMMON',
                                        'NCFDIV',
                                        'NCFX',
                                        'NCF',
                                        'SPS',
                                        'PAYOUTRATIO',
                                        'TANGIBLES',
                                        'TBVPS',
                                        'WORKINGCAPITAL',
                                        'price']
cmpny_financials_attr  = stck_fund_attr.copy()

cmpny_financials_attr.extend(['sp_price' ,'price_change' ,'sp_price_change' ,'diff' ,'status'])                                     
                  
cmpny_financials_attr = [x.lower() for x in cmpny_financials_attr]                                                                                   

df_company_fundmntls = pd.DataFrame(columns=stck_fund_attr)


In [57]:
df_company_fundmntls = quandl.get_table('SHARADAR/SF1', qopts={"columns":stck_fund_attr}, ticker=['MMM'],dimension = 'ARQ')

In [9]:
company_ticker = 'FB'
company_ticker

'FB'

In [10]:

try:
        
          print("starting " + company_ticker)

          df_company_fundmntls = quandl.get_table('SHARADAR/SF1', qopts={"columns":stck_fund_attr}, ticker=[company_ticker],dimension = 'ARQ')
        
          df_company_fundmntls = df_company_fundmntls.fillna(0.0)
            
          df_company_fundmntls = df_company_fundmntls[df_company_fundmntls.price != 0.0]
        
          df_company_fundmntls = df_company_fundmntls.reset_index(drop=True)
    
          df_company_fundmntls = df_company_fundmntls.sort_values(by=['datekey'])
          df_company_fundmntls["sp_price"] = 0.0
          df_company_fundmntls["price_change"] = 0.0
          df_company_fundmntls["sp_price_change"] = 0.0
          df_company_fundmntls["diff"] = 0.0
          df_company_fundmntls["status"] = "underperform"
            

    
          stock_start_value = 0.0
          sp_start_value = 0.0
          diff = 0
        

         
          
          for index,row in df_company_fundmntls.iterrows():
             
             

              try:
                   check_sp_date_exists = sp_test[(sp_test.datekey == row["datekey"])]
                   df_company_fundmntls.set_value(index,"sp_price",check_sp_date_exists["sp_price"])

              except ValueError:
                   rpt_dt = row["datekey"]
                   print(rpt_dt)
                   rpt_dt = rpt_dt - dateutil.relativedelta.relativedelta(days=4)
                   print(rpt_dt)
                   check_sp_date_exists = sp_test[(sp_test.datekey == rpt_dt)]
                   df_company_fundmntls.set_value(index,"sp_price",check_sp_date_exists["sp_price"])
                  

    
            
                    
              if (index != 0):

                  if (stock_start_value > 0):
                      df_company_fundmntls.set_value(index,"price_change",(((row["price"] - stock_start_value) / stock_start_value)*100))

                  if (sp_start_value > 0.0):

                      df_company_fundmntls.set_value(index,"sp_price_change" ,(((check_sp_date_exists["sp_price"] - sp_start_value) / sp_start_value)*100))
                  diff = (df_company_fundmntls.get_value(index,"price_change") - df_company_fundmntls.get_value(index,"sp_price_change"))
                  df_company_fundmntls.set_value(index,"diff", diff)
              else:
                  stock_start_value = row["price"]
                  sp_start_value = df_company_fundmntls.get_value(index,"sp_price")


              if diff > 0:
                    status = "outperform"
              else:
                    status = "underperform"  
            
              df_company_fundmntls.set_value(index,"status", status)   
#               stock_start_value = row["price"]

              

              
              
            
          print("ending" + company_ticker)

except quandl.NotFoundError as e:

        print(company_ticker + " not found ")
        pass
        
except Exception as e :
        print("Unexpected error:", sys.exc_info()[0])
        pass

starting FB
endingFB


/Users/jijijacob/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:36: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/Users/jijijacob/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:62: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/Users/jijijacob/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:70: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/Users/jijijacob/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:53: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/Users/jijijacob/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:57: FutureWarning: set_value is deprecated and wi

In [11]:

X_test = df_company_fundmntls[(df_company_fundmntls.datekey == df_company_fundmntls["datekey"].max())]
# X_test = X_test.reset_index()
column_not_needed = ["datekey","price","sp_price","price_change",'sp_price_change','diff','status']
X_test.drop(column_not_needed,axis=1,inplace=True)

/Users/jijijacob/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [103]:
X_test

,de,pe1,ps1,pb,netmargin,marketcap,ev,evebitda,revenueusd,gp,...,ncfdebt,ncfcommon,ncfdiv,ncfx,ncf,sps,payoutratio,tangibles,tbvps,workingcapital
22,0.137,35.171,13.779,7.545,0.329,5.609266e+11,5.528476e+11,23.407,12972000000,11361000000,...,0,-958000000,0,43000000,878000000,14.013,0.0,64419000000,22.137,44803000000


In [12]:
saved_rf = joblib.load('rand_frst.pkl')

In [13]:
saved_rf.predict(X_test)[0]

'underperform'

In [14]:
df_pred_val = df_company_fundmntls[['datekey','pe1','tangibles','sps','ps1','debtusd','evebitda',
                      'depamor','de','epsdil','price','sp_price','price_change','sp_price_change','status']]


In [15]:
df_pred_val["status"] = saved_rf.predict(X_test)[0]

/Users/jijijacob/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [17]:
df_pred_val = df_pred_val.reset_index()

In [18]:
df_pred_val

,index,datekey,pe1,tangibles,sps,ps1,debtusd,evebitda,depamor,de,epsdil,price,sp_price,price_change,sp_price_change,status
0,0,2012-07-31,0.000,14119000000,0.000,0.000,706000000,0.000,139000000,0.122,-0.08,21.71,1379.319946,0.000000,0.000000,underperform
1,1,2012-10-24,0.000,14615000000,0.000,0.000,902000000,0.000,176000000,0.132,-0.02,23.23,1408.750000,7.001382,2.133664,underperform
2,2,2013-02-01,1486.500,13715000000,2.537,11.719,2356000000,59.290,224000000,0.285,0.02,29.73,1513.170044,36.941502,9.704065,underperform
3,3,2013-05-02,1448.500,13662000000,2.301,12.593,2258000000,53.955,241000000,0.282,0.09,28.97,1597.589966,33.440811,15.824466,underperform
4,4,2013-07-25,143.167,14093000000,2.542,13.518,2167000000,30.751,222000000,0.273,0.13,34.36,1690.250000,58.268079,22.542272,underperform
5,5,2013-11-01,115.698,13324000000,2.828,17.592,575000000,37.897,274000000,0.144,0.17,49.75,1761.640015,129.157070,27.718012,underperform
6,6,2014-01-31,100.919,16173000000,3.253,19.235,476000000,41.008,274000000,0.157,0.21,62.57,1782.589966,188.208199,29.236873,underperform
7,7,2014-04-25,73.987,17346000000,3.504,16.471,392000000,31.971,264000000,0.137,0.25,57.71,1863.400024,165.822202,35.095561,underperform
8,8,2014-07-24,78.926,19097000000,3.912,19.168,326000000,35.266,257000000,0.132,0.30,74.98,1987.979980,245.370797,44.127545,underperform
9,9,2014-10-30,67.991,20259000000,4.330,17.117,278000000,32.888,289000000,0.139,0.30,74.11,1994.650024,241.363427,44.611120,underperform


In [71]:
df_pred_val

,index,datekey,pe1,tangibles,sps,ps1,debtusd,evebitda,depamor,de,epsdil,price,sp_price,price_change,sp_price_change,status
0,0,2012-07-31,0.000,14119000000,0.000,0.000,706000000,0.000,139000000,0.122,-0.08,21.71,1379.319946,0.000000,0.000000,underperform
1,1,2012-10-24,0.000,14615000000,0.000,0.000,902000000,0.000,176000000,0.132,-0.02,23.23,1408.750000,7.001382,2.133664,underperform
2,2,2013-02-01,1486.500,13715000000,2.537,11.719,2356000000,59.290,224000000,0.285,0.02,29.73,1513.170044,36.941502,9.704065,underperform
3,3,2013-05-02,1448.500,13662000000,2.301,12.593,2258000000,53.955,241000000,0.282,0.09,28.97,1597.589966,33.440811,15.824466,underperform
4,4,2013-07-25,143.167,14093000000,2.542,13.518,2167000000,30.751,222000000,0.273,0.13,34.36,1690.250000,58.268079,22.542272,underperform
5,5,2013-11-01,115.698,13324000000,2.828,17.592,575000000,37.897,274000000,0.144,0.17,49.75,1761.640015,129.157070,27.718012,underperform
6,6,2014-01-31,100.919,16173000000,3.253,19.235,476000000,41.008,274000000,0.157,0.21,62.57,1782.589966,188.208199,29.236873,underperform
7,7,2014-04-25,73.987,17346000000,3.504,16.471,392000000,31.971,264000000,0.137,0.25,57.71,1863.400024,165.822202,35.095561,underperform
8,8,2014-07-24,78.926,19097000000,3.912,19.168,326000000,35.266,257000000,0.132,0.30,74.98,1987.979980,245.370797,44.127545,underperform
9,9,2014-10-30,67.991,20259000000,4.330,17.117,278000000,32.888,289000000,0.139,0.30,74.11,1994.650024,241.363427,44.611120,underperform


In [99]:
df_company_fundmntls
df_company_fundmntls = df_company_fundmntls.reset_index(drop=True)

In [121]:
df_company_fundmntls

,datekey,de,pe1,ps1,pb,netmargin,marketcap,ev,evebitda,revenueusd,...,payoutratio,tangibles,tbvps,workingcapital,price,sp_price,price_change,sp_price_change,diff,status
0,2012-07-31,0.122,0.000,0.000,3.495,-0.133,4.651080e+10,4.511880e+10,0.000,1184000000,...,0.0,14119000000,7.514,10933000000,21.71,1379.319946,0.000000,0.000000,0.000000,underperform
1,2012-10-24,0.132,0.000,0.000,3.551,-0.047,5.032611e+10,4.875011e+10,0.000,1262000000,...,0.0,14615000000,6.039,11205000000,23.23,1408.750000,7.001382,2.133664,4.867718,outperform
2,2013-02-01,0.285,1486.500,11.719,6.025,0.027,7.082085e+10,7.079285e+10,59.290,1585000000,...,0.0,13715000000,5.782,10215000000,29.73,1513.170044,36.941502,9.704065,27.237437,outperform
3,2013-05-02,0.282,1448.500,12.593,5.924,0.149,7.004673e+10,6.997973e+10,53.955,1458000000,...,0.0,13662000000,5.726,9979000000,28.97,1597.589966,33.440811,15.824466,17.616344,outperform
4,2013-07-25,0.273,143.167,13.518,6.776,0.183,8.367759e+10,8.284359e+10,30.751,1813000000,...,0.0,14093000000,5.855,10341000000,34.36,1690.250000,58.268079,22.542272,35.725807,outperform
5,2013-11-01,0.144,115.698,17.592,9.360,0.209,1.221277e+11,1.196027e+11,37.897,2016000000,...,0.0,13324000000,5.483,9565000000,49.75,1761.640015,129.157070,27.718012,101.439058,outperform
6,2014-01-31,0.157,100.919,19.235,10.313,0.201,1.595374e+11,1.566904e+11,41.008,2586000000,...,0.0,16173000000,6.585,11970000000,62.57,1782.589966,188.208199,29.236873,158.971326,outperform
7,2014-04-25,0.137,73.987,16.471,8.849,0.255,1.481039e+11,1.454979e+11,31.971,2502000000,...,0.0,17346000000,6.816,13023000000,57.71,1863.400024,165.822202,35.095561,130.726640,outperform
8,2014-07-24,0.132,78.926,19.168,10.626,0.271,1.949519e+11,1.908939e+11,35.266,2910000000,...,0.0,19097000000,7.460,14343000000,74.98,1987.979980,245.370797,44.127545,201.243252,outperform
9,2014-10-30,0.139,67.991,17.117,9.728,0.250,2.066074e+11,1.978864e+11,32.888,3203000000,...,0.0,20259000000,7.831,14881000000,74.11,1994.650024,241.363427,44.611120,196.752307,outperform
